### This notebook is used for capstone project for IBM Data Science Professional certificate

In [112]:
import pandas as pd  
import numpy as np
import requests 
from bs4 import BeautifulSoup
import pprint

Make a dictionary of all Toronto Boroughs and Neighborhoods with Postal Codes

In [125]:

#neighborhoods_dict = {'M5G': [['Downtown Toronto'],['Central', 'Bay Street']]}

neighborhoods_dict = {}

#I can't control wikipedia, so I downloaded the page and put in my github repo

with open('List of postal codes of Canada_ M - Wikipedia.html') as html_file:
    soup = BeautifulSoup(html_file, 'lxml')
    
match = soup.find('table', class_='wikitable sortable jquery-tablesorter')
match = match.find('tbody')

table_rows = match.find_all('tr')


#process the table founded in the wiki page and make a dictionary of following form:
#{'Code': [['Borough'],['NeighborhoodS',...']]}
for item in table_rows:
    table_data = item.find_all('td')
        
    for i, data in enumerate(table_data):
        table_data[i] = data.text.rstrip()
  
    #process the table_data to dict: {'Code': [['Borough'],['NeighborhoodS',...']]}
    code = table_data[0]
    borough = table_data[1]
    neighborhood = table_data[2]
    
    if borough == 'Not assigned':  #filter all entries where borough is undefined
        pass
    elif code not in neighborhoods_dict: 
        if neighborhood != 'Not assigned':
            neighborhoods_dict[code]=[[borough],[neighborhood]]
        else:
            neighborhoods_dict[code]=[[borough],[borough]]
    elif code in neighborhoods_dict:
            neighborhoods_dict[code][1].append(neighborhood)
    else:
        print('something wrong with data') #check if something inconsistent in data
   

#### Let's transform our dictionary to panda's dataframe


In [126]:
# define the dataframe columns
column_names = ['PostalCode','Borough','Neighborhood']

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns = column_names)

#Take a look at the empty dataframe to confirm that the columns are as intended.
neighborhoods


,PostalCode,Borough,Neighborhood


In [155]:
for k, v in neighborhoods_dict.items():
 
    borough = v[0][0]
    Neighborhood = v[1]
  
    neighborhoods = neighborhoods.append({'PostalCode': k, 'Borough': borough, 'Neighborhood': Neighborhood} , ignore_index=True)

neighborhoods.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Harbourfront, Regent Park]"
3,M6A,North York,"[Lawrence Heights, Lawrence Manor]"
4,M7A,Queen's Park,[Queen's Park]
5,M9A,Etobicoke,[Islington Avenue]
6,M1B,Scarborough,"[Rouge, Malvern]"
7,M3B,North York,[Don Mills North]
8,M4B,East York,"[Woodbine Gardens, Parkview Hill]"
9,M5B,Downtown Toronto,"[Ryerson, Garden District]"


In [153]:
print(neighborhoods.shape)

(309, 3)
